## Calculate 15 providers daily revenue and market share in that revenue on Reed

In [1]:
# Import required modules
import pandas as pd
import numpy as np
#pd.set_option('display.max_rows', 500)

In [3]:
# Calculate revenue
def calculateRevenueAndUnitSold(preDay,nextDay):
    """Takes two dfs as argument. One is yesterday's data and the other is today's data"""
    preDf = pd.read_csv(f"{preDay}_15_providers.csv")
    nextDf = pd.read_csv(f"{nextDay}_15_providers.csv")
    
    # Drop all the enquired courses-previous day, since we're only interested in sold courses
    toDropPre = preDf[preDf.sold_or_enq.str.contains("enquired")].index
    preDf = preDf.drop(toDropPre,axis=0).reset_index(drop=True)
    
    # Drop all the enquired courses-next day
    toDropNext = nextDf[nextDf.sold_or_enq.str.contains("enquired")].index
    nextDf = nextDf.drop(toDropNext,axis=0).reset_index(drop=True)
    
    # Merge the two dataframes on ids
    mergedDf = pd.merge(nextDf,preDf, on="id", how="left",indicator=True)
    
    # Return data common on both dataframes
    mergedBothDf = mergedDf[mergedDf["_merge"]=="both"]
    
    # Column length to insert no of unit sold of course at the at of the dataframe
    columnLengthForUnitSold = len(mergedBothDf.columns.values)
    
    # Create a column name "unitSold" between two time duration at the the end of the mergedReqDf
    mergedBothDf.insert(loc=columnLengthForUnitSold,value=mergedBothDf.sold_x-mergedBothDf.sold_y, column="unitSold")
    
    # List to store actual revenue.
    # According to Reed, the revenue earned by a provider varies according to following price s
    actualRevenue = []
    totalMissingOriginalPriceNextDay = mergedBothDf[mergedBothDf.original_price_x.isna()].provider_x.value_counts().sum()
    totalMissingOriginalPricePreDay = mergedBothDf[mergedBothDf.original_price_y.isna()].provider_y.value_counts().sum()

    if totalMissingOriginalPriceNextDay<totalMissingOriginalPricePreDay:
        for prc,sale in zip(mergedBothDf.price_x,mergedBothDf.unitSold):
            if 10<=prc<100:
                actualRevenue.append(prc*0.4*sale)
            elif 100<=prc<150:
                actualRevenue.append(prc*0.55*sale)

            elif 150<=prc<200:
                actualRevenue.append(prc*0.6*sale)

            elif 200<=prc<300:
                actualRevenue.append(prc*0.7*sale)

            elif 300<=prc<500:
                actualRevenue.append(prc*0.75*sale)

            elif 500<=prc:
                actualRevenue.append(prc*0.8*sale)
    else:
        for prc,sale in zip(mergedBothDf.price_y,mergedBothDf.unitSold):
            if 10<=prc<100:
                actualRevenue.append(prc*0.4*sale)
            elif 100<=prc<150:
                actualRevenue.append(prc*0.55*sale)

            elif 150<=prc<200:
                actualRevenue.append(prc*0.6*sale)

            elif 200<=prc<300:
                actualRevenue.append(prc*0.7*sale)

            elif 300<=prc<500:
                actualRevenue.append(prc*0.75*sale)

            elif 500<=prc:
                actualRevenue.append(prc*0.8*sale)
        
    # Create a new colum named actual revenue at the end of the current df
    columnLengthForActualRevenue = len(mergedBothDf.columns.values)
    mergedBothDf.insert(loc=columnLengthForActualRevenue,value=actualRevenue,column="actualRevenue")
    
    # Group revenue by providers and sort by index (providers name). This is a series object with index name "provider"
    revenueGroupedByProviders =  mergedBothDf.groupby('provider_x').actualRevenue.sum().round().astype(int).sort_index()
    unitSoldGroupedByProviders = mergedBothDf.groupby('provider_x').unitSold.sum().round().astype(int).sort_index()
    
    # Convert the series to dataframe and make the "provider" column again
    revenueGroupedByProvidersDf = revenueGroupedByProviders.to_frame().reset_index(level="provider_x")
    unitSoldGroupedByProvidersDf = unitSoldGroupedByProviders.to_frame().reset_index(level="provider_x")
    
    # Make 2 columns named "individual share of revenue" and "individual share of unit sold"
    revenueGroupedByProvidersDf["indvShareRevenue"] = round(revenueGroupedByProvidersDf.actualRevenue/revenueGroupedByProvidersDf.actualRevenue.sum(),4)
    unitSoldGroupedByProvidersDf["indvShareUnitSold"] = round(unitSoldGroupedByProvidersDf.unitSold/unitSoldGroupedByProvidersDf.unitSold.sum(),4)
    
    # Set "provider" as index before transposing so that we can insert "date" column later
    revenueGroupedByProvidersDf = revenueGroupedByProvidersDf.set_index("provider_x")
    unitSoldGroupedByProvidersDf = unitSoldGroupedByProvidersDf.set_index("provider_x")
    
    # Transpost the data. It will create "provider" as column from index
    revenueGroupedByProvidersDfT = revenueGroupedByProvidersDf.T
    unitSoldGroupedByProvidersDfT = unitSoldGroupedByProvidersDf.T
    
    """
    1. After transposing, index 0 is the actualRevenue column, and index 1 is the indShareRevenue 
    column of revenueGroupedByProvidersDfT.
    2. And index 0 is the unitSold column, and index 1 in the indShareUnitSold column
    of unitSoldGroupedByProvidersDfT.
    """
    # Let's extract the required variables off those 2 dataframes
    # Extract actual revenue
    extractActualRevenue = revenueGroupedByProvidersDfT.iloc[[0]] # series to dataframe or, to_frame()
    
    # Extract unitSold
    extractUnitSold = unitSoldGroupedByProvidersDfT.iloc[[0]]
    
    # Extract individual share of revenue
    extractIndShareRevenue = revenueGroupedByProvidersDfT.iloc[[1]]
    
    # Extract individual share of unitSold
    extractIndShareUnitSold = unitSoldGroupedByProvidersDfT.iloc[[1]]
    
    # Insert date column in those 4 dataframes extracted above
    extractActualRevenue.insert(loc=0, value=preDay,column="date")
    extractUnitSold.insert(loc=0, value=preDay,column="date")
    extractIndShareRevenue.insert(loc=0, value=preDay,column="date")
    extractIndShareUnitSold.insert(loc=0, value=preDay,column="date")
    
    
    # Insert one more column each named "combined" to the 4 dataframes extracted above
    # Calculate combined (total of 5 brands) revenue and unit sold
    combinedRevenue = extractActualRevenue["Course Gate"]+extractActualRevenue["Euston College"]+extractActualRevenue["Janets"]+extractActualRevenue["One Education"]+extractActualRevenue["Training Express Ltd"]
    combinedUnitSold = extractUnitSold["Course Gate"]+extractUnitSold["Euston College"]+extractUnitSold["Janets"]+extractUnitSold["One Education"]+extractUnitSold["Training Express Ltd"]
    
    # Calculate combined revenue sahre and combined unit sale share 
    combinedShareRevenue = extractIndShareRevenue["Course Gate"]+extractIndShareRevenue["Euston College"]+extractIndShareRevenue["Janets"]+extractIndShareRevenue["One Education"]+extractIndShareRevenue["Training Express Ltd"]
    combinedShareUnitSold = extractIndShareUnitSold["Course Gate"]+extractIndShareUnitSold["Euston College"]+extractIndShareUnitSold["Janets"]+extractIndShareUnitSold["One Education"]+extractIndShareUnitSold["Training Express Ltd"]
    
    # Insert combined revenue and combined unit sale to the df
    extractActualRevenue.insert(loc=1,value=combinedRevenue.sum(),column="combined")
    extractUnitSold.insert(loc=1,value=combinedUnitSold.sum(),column="combined")
    
    # Insert combined revenue share and unit sold share to the df
    extractIndShareRevenue.insert(loc=1,value=combinedShareRevenue.sum(),column="combined")
    extractIndShareUnitSold.insert(loc=1,value=combinedShareUnitSold.sum(),column="combined")
    
    """first time we need to create a csv file, then we will be appending to those csv files per day"""
#     return extractIndShareRevenue.to_csv('revenueShare.csv',index=None),\
# extractIndShareUnitSold.to_csv('unitSoldShare.csv',index=None),\
# extractActualRevenue.to_csv("revenue.csv",index=None),\
# extractUnitSold.to_csv("unitSold.csv",index=None)
    return extractIndShareRevenue.to_csv('revenueShare.csv',mode="a",header=None,index=None),\
    extractIndShareUnitSold.to_csv("unitSoldShare.csv",index=None,mode="a",header=None),\
    extractActualRevenue.to_csv("revenue.csv",index=None,mode="a",header=None),\
    extractUnitSold.to_csv("unitSold.csv",index=None,mode="a",header=None)

#### Calculate revenue of 28 Jan. Run this cell onece
calculateRevenueAndUnitSold("28_Jan","29_Jan")

In [9]:
# Lets calculate daywise revenue and unit sale from 29 January to 12 March. And append the value.
# Don't run any cell more than once to prevent the append the value with same date more than once.
calculateRevenueAndUnitSold("17_Mar","18_Mar")

(None, None, None, None)

## Calculate average revenue and average market share after n days. These data is also used to make streamlit app that will be deployed to heroku.

In [14]:
def calculateAverageRevenueAndShare():
    """We are taking no arguments to the function>
    It creates 2 output files which will be used to make streamlit app to deploy the app on heroku."""
    
    # Read in revenue and revenue share data
    extractIndShareRevenue = pd.read_csv("revenueShare.csv")
    extractActualRevenue = pd.read_csv("revenue.csv")
    
    # Calculate percentage share. Multiply by 100 except date
    percentShare = pd.concat([extractIndShareRevenue.date, extractIndShareRevenue.iloc[:,1:]*100],axis=1)

    # Set date as index and transpose the df to insert new column "Average"
    # Now date is the column of makePercentT
    percentShareT = percentShare.set_index("date").T

    # Insert a column that calculates average revenue after n days
    # Insert position is the total number of columns. It inserts at the end of the df
    insertionPositionPercent = len(percentShareT.columns.values)
    percentShareT.insert(loc= insertionPositionPercent, value = np.round(percentShareT.mean(axis=1),2),column=f"Avg after {insertionPositionPercent} days")

    # Create avg revenue column
    averageRevenue = extractActualRevenue

    # Set date as index and transpose the df to insert new column "Average"
    # Now date is the column of careateAveT
    averageRevenueT = averageRevenue.set_index("date").T

    # Insert position is the total number of columns
    insertionPositionRev = len(averageRevenueT.columns.values)
    averageRevenueT.insert(loc= insertionPositionRev, value = np.round(averageRevenueT.mean(axis=1),2),column=f"Avg after {insertionPositionRev} days")

    # Round those columns values for streamlit plotting
    percentShareTR = percentShareT.round(2)
    averageRevenueTR = averageRevenueT.round()

    # Transpose back to save as csv and make date column again
    """round actualRevenueHeroku to zero and marketShareHeroku to two decimal points"""
    actualRevenueHeroku = averageRevenueTR.T.reset_index(level="date")
    marketShareHeroku = percentShareTR.T.reset_index(level="date")

    # overwrite the csv files with clean data
    return marketShareHeroku.to_csv("marketShareHeroku.csv",index=None),\
actualRevenueHeroku.to_csv("actualRevenueHeroku.csv",index=None)

In [15]:
# Call the function to create two csv files to make streamlit app
calculateAverageRevenueAndShare()

(None, None)